In [1]:
import sys

sys.path.append("../")

In [2]:
import argparse
import datetime
import json
import os
import re
from time import sleep

import numpy as np
import pandas as pd
import requests

from vook_db_v7.config import MAX_PAGE, REQ_URL, WANT_ITEMS, req_params

# from logzero import logger

In [12]:
brand = "リーバイス levis"
item = "501 66前期"

platform_id = 1
knowledge_id = 10

In [13]:
cnt = 1
keyword = f"{brand} {item} 中古"

req_params["page"] = cnt
req_params["keyword"] = keyword
df = pd.DataFrame(columns=WANT_ITEMS)

# ページループ
# logger.info("loop start!")
while True:
    req_params["page"] = cnt
    res = requests.get(REQ_URL, req_params)
    res_code = res.status_code
    res = json.loads(res.text)
    if res_code != 200:
        print(
            f"""
        ErrorCode -> {res_code}\n
        Error -> {res['error']}\n
        Page -> {cnt}"""
        )
    else:
        if res["hits"] == 0:
            print("返ってきた商品数の数が0なので、ループ終了")
            break
        tmp_df = pd.DataFrame(res["Items"])[WANT_ITEMS]
        df = pd.concat([df, tmp_df], ignore_index=True)
    if cnt == MAX_PAGE:
        print("MAX PAGEに到達したので、ループ終了")
        break
    # logger.info(f"{cnt} end!")
    cnt += 1
    # リクエスト制限回避
    sleep(1)

    print("Finished!!")

Finished!!
Finished!!
返ってきた商品数の数が0なので、ループ終了


In [14]:
df["platform_id"] = platform_id
df["knowledge_id"] = knowledge_id
df["size_id"] = 39
df_prev = pd.read_csv("../data/output/products_raw_prev.csv")
PREV_ID_MAX = df_prev["id"].max()
df["id"] = np.arange(PREV_ID_MAX, PREV_ID_MAX + len(df)) + 1

df_main = df.rename(
    columns={"itemName": "name", "itemPrice": "price", "itemUrl": "url"}
)
df_main = df_main.reindex(
    columns=["id", "name", "url", "price", "knowledge_id", "platform_id", "size_id"]
)

run_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
df_main["created_at"] = run_time
df_main["updated_at"] = run_time

In [15]:
df_main['size_id'].dtype

dtype('int64')

In [16]:
df_main['price'].dtype


dtype('O')

In [18]:
df_main

,id,name,url,price,knowledge_id,platform_id,size_id,created_at,updated_at
0,104,Levi's リーバイス デニムパンツ ヴィンテージ 501 66前期 ビックE トップボタ...,https://hb.afl.rakuten.co.jp/hgc/g00rc6oe.brn1...,418979,10,1,39,2023-10-04 21:57:13.428304,2023-10-04 21:57:13.428304
1,105,リーバイス 70s 501 66前期 BIGE ストレート デニムパンツ W31 Levi'...,https://hb.afl.rakuten.co.jp/hgc/g00rf2ie.brn1...,327800,10,1,39,2023-10-04 21:57:13.428304,2023-10-04 21:57:13.428304
2,106,Levi's ヴィンテージ リーバイス パンツ 501 66前期 BigE デニムパンツ 濃...,https://hb.afl.rakuten.co.jp/hgc/g00s4pxe.brn1...,251800,10,1,39,2023-10-04 21:57:13.428304,2023-10-04 21:57:13.428304
3,107,リーバイス 501 66前期 赤耳 テーパード デニム パンツ W32 Levi's メンズ...,https://hb.afl.rakuten.co.jp/hgc/g00rf2ie.brn1...,96800,10,1,39,2023-10-04 21:57:13.428304,2023-10-04 21:57:13.428304
4,108,【中古】リーバイス Levis 70S 66前期 501 裏刻印6 BIG E 赤耳 Sin...,https://hb.afl.rakuten.co.jp/hgc/g00rbope.brn1...,91289,10,1,39,2023-10-04 21:57:13.428304,2023-10-04 21:57:13.428304
5,109,LEVIS VINTAGE 70s 501 66前期 INDIGO 85% リーバイス ヴィ...,https://hb.afl.rakuten.co.jp/hgc/g00s3vhe.brn1...,89100,10,1,39,2023-10-04 21:57:13.428304,2023-10-04 21:57:13.428304
6,110,Levi's リーバイス ヴィンテージ オリジナル 501 66前期 Big E 両面不均等...,https://hb.afl.rakuten.co.jp/hgc/g00rc6oe.brn1...,88902,10,1,39,2023-10-04 21:57:13.428304,2023-10-04 21:57:13.428304
7,111,"Levi's リーバイス デニム、ジーンズ パンツ Pants, Trousers Deni...",https://hb.afl.rakuten.co.jp/hgc/g00u1a6e.brn1...,88000,10,1,39,2023-10-04 21:57:13.428304,2023-10-04 21:57:13.428304
8,112,LEVIS VINTAGE 70s 501 66前期 スモールe バッチ無 リーバイス ヴィ...,https://hb.afl.rakuten.co.jp/hgc/g00s3vhe.brn1...,83600,10,1,39,2023-10-04 21:57:13.428304,2023-10-04 21:57:13.428304
9,113,リーバイス 501 ネイビー LEVI'S 66前期 BIGE 初期 Vステッチ ウエストシ...,https://hb.afl.rakuten.co.jp/hgc/g00tiqde.brn1...,82500,10,1,39,2023-10-04 21:57:13.428304,2023-10-04 21:57:13.428304


In [62]:
file_name = "products_raw"
df_main.to_csv("../data/output/" + file_name + ".csv", index=False)